This notebook was used during development to incrementally test the program for task 3

### Library

In [22]:
import psycopg2
import time
import itertools

### Constants

In [23]:
TABLE = 'task_1'
VALID_FDS = {}

### Helper Methods

In [24]:
def create_query_string(LHS=[], RHS=[], table=TABLE):
    "creates a the query that will return whether RHS has a functional dependency on LHS."
    if type(LHS) == str : 
        LHS = [LHS]
    if type(RHS) == str : 
        RHS = [RHS]
    
    
    LHS_string = ''
    for lhs in LHS:
        LHS_string += lhs + ', '
    LHS_string = LHS_string[:-2]

    RHS_string_1 = ''
    RHS_string_2 = ''
    RHS_string_3 = ''
    for rhs in RHS:
        RHS_string_1 += f'COUNT({rhs}), '
        RHS_string_2 += rhs + ', '
        RHS_string_3 += f'COUNT({rhs}) > 1 AND '
    
    RHS_string_1 = RHS_string_1[:-2]
    RHS_string_2 = RHS_string_2[:-2]
    RHS_string_3 = RHS_string_3[:-5]

    query_string = f'SELECT EXISTS (SELECT {LHS_string}, {RHS_string_1} FROM (SELECT DISTINCT {LHS_string}, {RHS_string_2} FROM {table}) AS SQ GROUP BY {LHS_string} HAVING {RHS_string_3});'
    return query_string


def find_fds_with_pruning (LHS, RHS, cur, conn, table=TABLE) :
    prune = False
    if type(RHS) == list:
        RHS = RHS[0]
    if RHS in VALID_FDS.keys():
        for lhs in LHS:
            if lhs in VALID_FDS[RHS]:
                prune = True
                print(f'{LHS} -> {RHS} PRUNNED because FD {lhs}->{RHS} exists')
    
    if (not prune): 
        start_time = time.time()
        cur.execute(create_query_string(LHS=LHS, RHS=RHS, table=table))
        conn.commit()
        duration = round(time.time() - start_time,3)
        result = cur.fetchall()
        if (result[0][0]):
            print(f"Functional Dependency ABSENT in : {LHS} -> {RHS}, Duration: {duration}")
        else:
            print(f"Functional Dependency PRESENT in : {LHS} -> {RHS}, Duration: {duration}")
            if RHS not in VALID_FDS.keys():
                VALID_FDS[RHS] = set()
            for lhs in LHS:
                    VALID_FDS[RHS].add(lhs)

### PSQL

#### Connection

In [25]:
conn = psycopg2.connect("dbname=csci_620_ass_2")
print("Database connected successfully")
cur = conn.cursor()
conn.commit()

Database connected successfully


#### Get Attributes

In [26]:
cur.execute(f"Select * FROM {TABLE} LIMIT 0")
attributes = [desc[0] for desc in cur.description]

### testing method

In [28]:
find_fds_with_pruning (LHS=['genreId'], RHS=['genre'], cur=cur, conn=conn)
find_fds_with_pruning (LHS=['movieId'], RHS=['genre'], cur=cur, conn=conn)
find_fds_with_pruning (LHS=['genreId', 'movieId'], RHS=['genre'], cur=cur, conn=conn)

Functional Dependency PRESENT in : ['genreId'] -> genre, Duration: 1.62
Functional Dependency ABSENT in : ['movieId'] -> genre, Duration: 7.857
['genreId', 'movieId'] -> genre PRUNNED because FD genreId->genre exists


### All with max 2 LHS

In [29]:
start_time = time.time()
for RHS_n_LHS_quantity in range(2, 4):
    for subset in itertools.combinations(attributes, RHS_n_LHS_quantity):
        RHS_LHS = list(subset)
        RHS = RHS_LHS[0]
        LHS = RHS_LHS[1:]
        # print(f"Investigating: {LHS} -> {RHS}")
        find_fds_with_pruning(LHS=LHS, RHS=RHS, cur=cur, conn=conn)
        # find_fds(LHS=LHS, RHS=RHS, cur=cur, conn=conn)

print(f'Time the operation took: {round(time.time()-start_time,2)}')

Functional Dependency ABSENT in : ['type'] -> movieid, Duration: 3.707
Functional Dependency ABSENT in : ['startyear'] -> movieid, Duration: 3.031
Functional Dependency ABSENT in : ['runtime'] -> movieid, Duration: 2.751
Functional Dependency ABSENT in : ['avgrating'] -> movieid, Duration: 4.202
Functional Dependency ABSENT in : ['genreid'] -> movieid, Duration: 4.645
Functional Dependency ABSENT in : ['genre'] -> movieid, Duration: 4.156
Functional Dependency ABSENT in : ['memberid'] -> movieid, Duration: 5.117
Functional Dependency ABSENT in : ['birthyear'] -> movieid, Duration: 2.428
Functional Dependency ABSENT in : ['character'] -> movieid, Duration: 6.451
Functional Dependency ABSENT in : ['startyear'] -> type, Duration: 0.658
Functional Dependency ABSENT in : ['runtime'] -> type, Duration: 0.696
Functional Dependency ABSENT in : ['avgrating'] -> type, Duration: 0.979
Functional Dependency ABSENT in : ['genreid'] -> type, Duration: 0.636
Functional Dependency ABSENT in : ['genre'